<a href="https://colab.research.google.com/github/DorMm1/Ono-Image-Processing/blob/master/Part_B_of_Final_Project_of_Image_Processing_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project Image Processing, Ono College - B.Sc Computer Science**
## Google Colab Notebook 3/3

### Students:
- StudentName = 'Dor Menahem', StudentID = '318687746', StudentMobile = '052-8028427'
- StudentName = 'Nir Barash', StudentID = '207541434', StudentMobile = '054-3172167'



In [1]:
import cv2
import numpy as np
import random
import tempfile
import shutil
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive
import ast



In [2]:
from google.colab import drive

drive.flush_and_unmount()

def setup_google_drive(dataset_folder="/content/drive/My Drive/Part_B_DataSet"):
    # Mount Google Drive
    drive.mount("/content/drive", force_remount=True)
    # Define paths to train and test folders
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    return train_folder, test_folder

# Example usage
train_folder, test_folder = setup_google_drive()
print("Train folder:", train_folder)
print("Test folder:", test_folder)


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Train folder: /content/drive/My Drive/Part_B_DataSet/train
Test folder: /content/drive/My Drive/Part_B_DataSet/test


In [8]:
import ast
import os

class Metadata:
    def __init__(self, shapes, line):
        self.shapes = shapes
        self.line = line

    def __iter__(self):
        # Iterate over the shapes attribute
        return iter(self.shapes)

def parse_metadata(metadata_text):
    # Extract shape metadata
    shape_metadata = metadata_text['shape_metadata']
    line_metadata = metadata_text['line_metadata']

    # Create a Metadata object
    parsed_metadata = Metadata(shape_metadata, line_metadata)

    return parsed_metadata

def load_metadata(dataset_folder, num_images):
    """
    Load metadata for each image from the .txt files.

    Args:
    - dataset_folder (str): Path to the dataset folder.
    - num_images (int): Total number of images in the dataset.

    Returns:
    - list: List containing Metadata objects for each image.
    """
    metadata = []
    for i in range(num_images):
        # Load metadata from the corresponding .txt file
        txt_filename = f"{i}.jpg.txt"
        txt_filepath = os.path.join(dataset_folder, txt_filename)
        with open(txt_filepath, "r") as f:
            metadata_text = ast.literal_eval(f.read())
            metadata.append(parse_metadata(metadata_text))
    return metadata

num_images = 5600
metadata = load_metadata(train_folder, num_images)

In [9]:
print("Metadata example Metadata objects:")
print(metadata[0].shapes)
print(metadata[0].line)
print("Amount of metadata objects:")
print(len(metadata))


Metadata example Metadata objects:
[{'shape_type': 'half_circle', 'color': 165, 'size': 10, 'position': [112, 59]}, {'shape_type': 'triangle', 'color': 97, 'size': 18, 'position': [57, 41]}]
{'shape_type': 'line', 'color': 14, 'a': 1.65625, 'b': -124.0}
Amount of metadata objects:
5600


In [10]:
def process_image(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to 128x128 -- image was generated 128X128 but never the less
    image = cv2.resize(image, (128, 128))
    # Normalize the image
    image = image / 255.0
    return image

def get_label(metadata):
    # Check if shapes length is 1 and shape is not a circle
    if len(metadata.shapes) == 1 and metadata.shapes[0]['shape_type'] != 'circle':
        return 1
    else:
        return 0

def create_data_tuples(image_directory, metadata_objects):
    data_tuples = []
    for i, metadata in enumerate(metadata_objects):
        # Construct the image filename
        image_filename = f"{i}.jpg"
        image_path = os.path.join(image_directory, image_filename)
        # Check if the image file exists
        if os.path.exists(image_path):
            # Load and preprocess the image
            image = process_image(image_path)
            # Get the label
            label = get_label(metadata)
            # Append the tuple (image, metadata, label) to the data list
            data_tuples.append((image, metadata, label))
    return data_tuples


# Example usage:
image_directory = train_folder
metadata_objects = metadata  # List of metadata objects

# Generate data tuples
data_tuples = create_data_tuples(image_directory, metadata_objects)

# Example of accessing the first data tuple
image, metadata, label = data_tuples[0]
print("Image size:", image.shape)
print("Label:", label)



Image size: (128, 128)
Label: 0


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define your neural network architecture
def create_model():
    # Image input
    image_input = layers.Input(shape=(128, 128, 1))
    # Metadata input
    metadata_input = layers.Input(shape=(len(metadata_features)+1,))

    # Image processing layers
    conv1 = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
    maxpool1 = layers.MaxPooling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu')(maxpool1)
    maxpool2 = layers.MaxPooling2D((2, 2))(conv2)
    flatten1 = layers.Flatten()(maxpool2)

    # Concatenate image and metadata features
    concat = layers.concatenate([flatten1, metadata_input])

    # Dense layers for classification
    dense1 = layers.Dense(64, activation='relu')(concat)
    output = layers.Dense(1, activation='sigmoid')(dense1)

    model = models.Model(inputs=[image_input, metadata_input], outputs=output)
    return model

# train_data: list of tuples (image, metadata, label)
train_data = data_tuples
# test_data: list of tuples (image, metadata, label)
metadata_objects_test = load_metadata(test_folder, 1400)
test_data = create_data_tuples(test_folder, metadata_objects_test)


In [43]:
# Define metadata features
metadata_features = ['color', 'size', 'position']

# Extract features and labels from train and test data
def extract_features(metadata):
    if metadata.shapes:  # Check if shapes list is not empty
        # Extract numerical features from metadata
        color = metadata.shapes[0]['color']
        size = metadata.shapes[0]['size']
        position_x, position_y = metadata.shapes[0]['position']
        # Return extracted features as a list
        return [color, size, position_x, position_y]
    else:
        return [0, 0, 0, 0]  # Return default values


X_train_images = np.array([data[0] for data in train_data])
X_train_metadata = np.array([extract_features(data[1]) for data in train_data])  # Assuming a function extract_features() extracts numerical features from Metadata
y_train_labels = np.array([data[2] for data in train_data])

X_test_images = np.array([data[0] for data in test_data])
X_test_metadata = np.array([extract_features(data[1]) for data in test_data])  # Assuming a function extract_features() extracts numerical features from Metadata
y_test_labels = np.array([data[2] for data in test_data])

# Normalize image data
X_train_images = X_train_images / 255.0
X_test_images = X_test_images / 255.0

In [44]:
# Define your model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train_images, X_train_metadata], y_train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate([X_test_images, X_test_metadata], y_test_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
140/140 [==============================] - 108s 766ms/step - loss: 0.3131 - accuracy: 0.9190 - val_loss: 0.3006 - val_accuracy: 0.9098
Epoch 2/10
140/140 [==============================] - 100s 716ms/step - loss: 0.2890 - accuracy: 0.9192 - val_loss: 0.2969 - val_accuracy: 0.9098
Epoch 3/10
140/140 [==============================] - 103s 735ms/step - loss: 0.2753 - accuracy: 0.9192 - val_loss: 0.2911 - val_accuracy: 0.9098
Epoch 4/10
140/140 [==============================] - 103s 739ms/step - loss: 0.2782 - accuracy: 0.9192 - val_loss: 0.2922 - val_accuracy: 0.9098
Epoch 5/10
140/140 [==============================] - 104s 743ms/step - loss: 0.2713 - accuracy: 0.9192 - val_loss: 0.2901 - val_accuracy: 0.9098
Epoch 6/10
140/140 [==============================] - 102s 725ms/step - loss: 0.2714 - accuracy: 0.9192 - val_loss: 0.2950 - val_accuracy: 0.9098
Epoch 7/10
140/140 [==============================] - 104s 739ms/step - loss: 0.2731 - accuracy: 0.9192 - val_loss: 0.2920 -